In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import annotate
import n_1rxi as method
import plot_utils

In [2]:
chunk_size = 20

In [3]:
fname = method.__file__.split('/')[-1][:-3].replace('x', str(chunk_size))
fname

'n_1r20i'

In [4]:
reference = pd.read_excel(f'gpt-ratings/{fname}.xlsx')

In [5]:
reference

,Unnamed: 0,Participant #,Test_Date,Question id,Question_en,Answer,Novelty,Feasibility,Value,W_Novelty,...,value_explanation,value,novelty_raw,novelty_idea_explanation,novelty_explanation,novelty,feasibility_raw,feasibility_idea_explanation,feasibility_explanation,feasibility
0,1629,64,2022-12-22T03:56:44,9,Wood chopsticks,ドラムスティック,1554,1656,1622,5,...,Existing products: Drumsticks are usually pric...,1500,1. The idea is to use wooden chopsticks as dru...,1. The idea is to use wooden chopsticks as dru...,"Comparing to other unconventional uses, this i...",30,1. Idea: Using wooden chopsticks as drumsticks...,1. Idea: Using wooden chopsticks as drumsticks.,Challenges: The size and weight of chopsticks ...,40
1,566,28,2022-04-04T03:11:22,26,Credit card,用紙,1640,1561,1553,13,...,"Existing products: Regular paper, which costs ...",1,1. Idea: Using a credit card as paper.\nThis i...,1. Idea: Using a credit card as paper.,"This idea is somewhat novel, as credit cards a...",40,1. Idea: Using a credit card as paper.\nFeasib...,1. Idea: Using a credit card as paper.,Feasibility: This is not feasible because cred...,10
2,1236,51,2022-11-04T04:11:38,6,PET bottle,武器,1624,1592,1616,9,...,Existing products: Non-lethal self-defense wea...,2000,1. Idea: Using a plastic bottle as a weapon.\n...,1. Idea: Using a plastic bottle as a weapon.,"This idea is somewhat novel, as plastic bottle...",40,1. Idea: Using a plastic bottle as a weapon.\n...,1. Idea: Using a plastic bottle as a weapon.,Feasibility: This is not very feasible as plas...,20
3,209,16,2022-02-17T02:55:14,29,Coin,食べる,1632,1505,1584,10,...,Existing products: Edible gold leaf used in lu...,1000,1. Idea: Using a coin as food.\nThis idea is q...,1. Idea: Using a coin as food.,This idea is quite novel as coins are not typi...,70,1. Idea: Using a coin as food.\nFeasibility: T...,1. Idea: Using a coin as food.,Feasibility: This is not feasible as coins are...,1
4,562,28,2022-04-04T03:11:22,22,Marbles,消毒,1671,1560,1585,12,...,Existing products for disinfection like saniti...,1000,1. Using marbles for disinfection\nThis idea i...,1. Using marbles for disinfection,This idea is quite unique as marbles are not t...,85,1. Idea: Using marbles for disinfection.\nThis...,1. Idea: Using marbles for disinfection.,This idea may not be feasible as marbles do no...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,190,16,2022-02-17T02:55:14,10,Drinking straw,投げる,1598,1586,1528,8,...,Existing products: Basic throwing toys can be ...,500,20. Using a straw to throw\nThe idea is to use...,20. Using a straw to throw,The idea is to use a straw to throw. This is n...,5,20. Using a straw to throw\nThis idea is very ...,20. Using a straw to throw,This idea is very feasible. Straws are lightwe...,100
1218,1592,63,2022-12-15T04:14:11,2,Ketchup,リップ,1576,1609,1601,8,...,Existing products: Lipstick can range from 500...,5000,20. Idea: Using ketchup as lipstick.\nThis is ...,20. Idea: Using ketchup as lipstick.,"This is a highly novel idea, as ketchup is not...",85,20. Idea: Using ketchup as lipstick.\nFeasibil...,20. Idea: Using ketchup as lipstick.,Feasibility: This is somewhat feasible. Ketchu...,30
1219,186,16,2022-02-17T02:55:14,6,PET bottle,舟,1609,1590,1608,8,...,Existing products: Toy boats can cost around 1...,1000,20. The idea is to use a pet bottle as a boat....,20. The idea is to use a pet bottle as a boat.,"This idea is quite novel, as it's not common t...",85,20. Idea: Using a plastic bottle as a boat.\nF...,20. Idea: Using a plastic bottle as a boat.,Feasibility: This is somewhat feasible. A plas...,50
1220,526,27,2022-03-31T06:32:49,16,Metal keys,スクラッチ,1552,1645,1616,5,...,Existing products: A scratch-off tool costs ar...,300,20. Idea: Using a metal key to scratch.\nThis ...,20. Idea: Using a metal key to scratch.,"This is a fairly common use of keys, especiall...",20,20. Idea: Use a metal key to scratch.\nFeasibi...,20. Idea: Use a metal key to scratch.,"Feasi

In [6]:
df1 = pd.read_excel('RAT_20231208_cleanBasicInfo.xlsx', sheet_name='AUT_Q1')
df2 = pd.read_excel('RAT_20231208_cleanBasicInfo.xlsx', sheet_name='AUT_Q2')
df_all = pd.concat([df1, df2])
df_all

,Participant #,Item_no,Item_name,Answer,Category_No,Category_contents
0,2021-07-01T03:13:45 b3e536-25f-013-261b-27c888...,26,Credit card,NaN,NaN,NaN
1,2021-07-01T03:13:45 b3e536-25f-013-261b-27c888...,26,Credit card,バターナイフ,448.0,cn:用来切蛋糕或是拆信 en:Used to cut cakes or open lett...
2,2021-07-01T03:13:45 b3e536-25f-013-261b-27c888...,26,Credit card,皿,780.0,cn:将蜡圈在纸上撒一圈，把里面当作盘子 en:Sprinkle wax in a circ...
3,2021-07-01T03:13:45 b3e536-25f-013-261b-27c888...,26,Credit card,しおり,178.0,cn:书签 en:Bookmark jp:しおり tw:書籤
4,2021-07-01T04:47:06 21a2df6-c6bc-87c6-657-dcbd...,19,Tape,NaN,NaN,NaN
...,...,...,...,...,...,...
1409,2023-12-01T04:30:01 53e64ee-84f-81cc-0c36-8d2b...,4,Newspaper,マスク,353.0,cn:可临时遮住口鼻 en:Can temporarily cover nose and m...
1410,2023-12-01T04:31:19 10a66f2-e66a-dd40-8b15-ca4...,22,Marbles,NaN,NaN,NaN
1411,2023-12-01T04:31:19 10a66f2-e66a-dd40-8b15-ca4...,22,Marbles,鉄琴のバチの先,2770.0,cn:像鼓槌一样敲鼓 en:Hit the drum as a drumstick jp:バ...
1412,2023-12-01T04:31:19 10a66f2-e66a-dd40-8b15-ca4...,22,Marbles,窓ガラス,2463.0,cn:比较小颗的玻璃珠 en:Smaller glass beads jp:小さいガラスビー...


In [7]:
df_new = pd.merge(
    df_all[['Item_no', 'Item_name', 'Answer']].rename(columns={'Item_no': 'Question id'}),
    reference[['Question id', 'Question']].drop_duplicates(),
    on='Question id').dropna().drop_duplicates().reset_index(drop=True)
df_new

,Question id,Item_name,Answer,Question
0,26,Credit card,バターナイフ,クレジットカード
1,26,Credit card,皿,クレジットカード
2,26,Credit card,しおり,クレジットカード
3,26,Credit card,ナイフ,クレジットカード
4,26,Credit card,卓球ラケット,クレジットカード
...,...,...,...,...
4245,21,Ball pen,ヘンゼルとグレーテルのパン,ボールペン
4246,21,Ball pen,ねじ回し,ボールペン
4247,21,Ball pen,あやし道具,ボールペン
4248,21,Ball pen,料理器具,ボールペン


In [8]:
needs_eval = pd.merge(df_new, reference, on=['Question id', 'Answer'], how='left', indicator=True)['_merge'] == 'left_only'

In [9]:
df_eval = df_new[needs_eval]
df_eval

,Question id,Item_name,Answer,Question
0,26,Credit card,バターナイフ,クレジットカード
1,26,Credit card,皿,クレジットカード
2,26,Credit card,しおり,クレジットカード
3,26,Credit card,ナイフ,クレジットカード
4,26,Credit card,卓球ラケット,クレジットカード
...,...,...,...,...
4245,21,Ball pen,ヘンゼルとグレーテルのパン,ボールペン
4246,21,Ball pen,ねじ回し,ボールペン
4247,21,Ball pen,あやし道具,ボールペン
4248,21,Ball pen,料理器具,ボールペン


In [16]:
df_reuse = df_new[~needs_eval].merge(
    reference[['Question id', 'Answer'] + [dim + suffix for dim in ('novelty', 'feasibility', 'value') for suffix in ('_raw', '_idea_explanation', '_explanation', '')]],
    on=['Question id', 'Answer'])
df_reuse

,Question id,Item_name,Answer,Question,novelty_raw,novelty_idea_explanation,novelty_explanation,novelty,feasibility_raw,feasibility_idea_explanation,feasibility_explanation,feasibility,value_raw,value_idea_explanation,value_explanation,value
0,26,Credit card,定規,クレジットカード,14. Idea: Using a credit card as a ruler.\nThi...,14. Idea: Using a credit card as a ruler.,"This idea is not particularly novel, as the st...",20,14. Idea: Using a credit card as a ruler.\nFea...,14. Idea: Using a credit card as a ruler.,Feasibility: This is highly feasible. Credit c...,80,14. Idea: Using a credit card as a ruler.\nExi...,14. Idea: Using a credit card as a ruler.,Existing products: Rulers can be bought for ar...,200
1,26,Credit card,カッター,クレジットカード,10. Idea: Using a credit card as a cutter.\nTh...,10. Idea: Using a credit card as a cutter.,"This idea is not very novel, as credit cards a...",30,10. Idea: Using a credit card as a cutter.\nFe...,10. Idea: Using a credit card as a cutter.,Feasibility: This is somewhat feasible as the ...,40,10. Idea: Using a credit card as a cutter.\nEx...,10. Idea: Using a credit card as a cutter.,"Existing products: A box cutter, which costs a...",500
2,26,Credit card,トランプ,クレジットカード,4. Idea: Using a credit card as a playing card...,4. Idea: Using a credit card as a playing card.,"This idea is somewhat novel, as credit cards a...",50,4. Idea: Using a credit card as a playing card...,4. Idea: Using a credit card as a playing card.,Feasibility: This is not very feasible as cred...,20,4. Idea: Using a credit card as a playing card...,4. Idea: Using a credit card as a playing card.,"Existing products: A deck of playing cards, wh...",500
3,26,Credit card,名札,クレジットカード,19. Idea: Using a credit card as a name tag.\n...,19. Idea: Using a credit card as a name tag.,"This is a fairly novel idea, as it is not a co...",60,19. Idea: Using a credit card as a name tag.\n...,19. Idea: Using a credit card as a name tag.,Feasibility: This is feasible. One could write...,60,19. Idea: Using a credit card as a name tag.\n...,19. Idea: Using a credit card as a name tag.,Existing products: Name tags can be bought for...,200
4,26,Credit card,調理器具,クレジットカード,15. Idea: Using a credit card as a cooking ute...,15. Idea: Using a credit card as a cooking ute...,"This idea is quite novel, as credit cards are ...",70,15. Idea: Using a credit card as a cooking ute...,15. Idea: Using a credit card as a cooking ute...,Feasibility: This is not feasible as a credit ...,5,15. Idea: Using a credit card as a cooking ute...,15. Idea: Using a credit card as a cooking ute...,"Existing products: A spatula, which costs arou...",500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1,Umbrella,水瓶,傘,15. Idea: Using an umbrella as a water bottle....,15. Idea: Using an umbrella as a water bottle.,"This is a highly novel idea, as umbrellas are ...",95,15. Idea: Using an umbrella as a water bottle....,15. Idea: Using an umbrella as a water bottle.,Feasibility: This is not feasible as an umbrel...,5,15. Idea: Use an umbrella as a water bottle.\n...,15. Idea: Use an umbrella as a water bottle.,Existing products: Water bottles are priced ar...,1000
279,1,Umbrella,パラシュート,傘,3. Using an umbrella as a parachute\nThis idea...,3. Using an umbrella as a parachute,"This idea is moderately novel, as umbrellas ar...",70,3. Using an umbrella as a parachute\nFeasibili...,3. Using an umbrella as a parachute,Feasibility: This is not feasible as an umbrel...,2,3. Idea: Using an umbrella as a parachute.\nEx...,3. Idea: Using an umbrella as a parachute.,Existing products: Parachutes can cost from 10...,50000
280,21,Ball pen,カッター,ボールペン,18. Idea: Using a ballpoint pen as a cutter.\n...,18. Idea: Using a ballpoint pen as a cutter.,"This is a somewhat novel idea, as pens are not...",60,18. Idea: Using a ballpoint pen as a cutter.\n...,18. Idea: Using a ballpoint pen as a cutter.,Feasibility: This is not feasible as a ballpoi...,5,18. Idea: Using a ballpoint pen as a cutter

In [11]:
import rate
rate.setkey(0)

In [12]:
df_eval = df_eval.sample(frac=1, random_state=8)

In [13]:
await method.rate(df_eval, chunk_size=chunk_size)

Request 1 of 642
Request 2 of 642
Request 3 of 642
Request 4 of 642
Request 5 of 642
Request 6 of 642
Request 7 of 642
Request 8 of 642
Request 9 of 642
Request 10 of 642
Request 11 of 642
Request 12 of 642
Request 13 of 642
Request 14 of 642
Request 15 of 642
Request 16 of 642
Request 17 of 642
Request 18 of 642
Request 19 of 642
Request 20 of 642
Request 21 of 642
Request 22 of 642
Request 23 of 642
Request 24 of 642
Request 25 of 642
Request 26 of 642
Request 27 of 642
Request 28 of 642
Request 29 of 642
Request 30 of 642
Request 31 of 642
Request 32 of 642
Request 33 of 642
Request 34 of 642
Request 35 of 642
Request 36 of 642
Request 37 of 642
Request 38 of 642
Request 39 of 642
Request 40 of 642
Request 41 of 642
Request 42 of 642
Request 43 of 642
Request 44 of 642
Request 45 of 642
Request 46 of 642
Request 47 of 642
Request 48 of 642
Request 49 of 642
Request 50 of 642
Request 51 of 642
Request 52 of 642
Request 53 of 642
Request 54 of 642
Request 55 of 642
Request 56 of 642
R

In [14]:
df_eval.to_excel('gpt-ratings/df_eval.xlsx')

In [15]:
df_eval

,Question id,Item_name,Answer,Question,value_raw,value_idea_explanation,value_explanation,value,feasibility_raw,feasibility_idea_explanation,feasibility_explanation,feasibility,novelty_raw,novelty_idea_explanation,novelty_explanation,novelty
169,19,Tape,カバー,テープ,1. Idea: Using tape as a cover\nExisting produ...,1. Idea: Using tape as a cover,Existing products like plastic covers or cloth...,1.000000e+03,1. Idea: Using tape as a cover.\nFeasibility: ...,1. Idea: Using tape as a cover.,Feasibility: This is feasible as tape can be u...,85.0,1. Use tape as a cover\nUsing tape as a cover ...,1. Use tape as a cover,Using tape as a cover is not a very novel idea...,20.0
739,10,Drinking straw,電線,ストロー,1. Idea: Using a straw as a wire.\nExisting pr...,1. Idea: Using a straw as a wire.,Existing products: Electrical wires are priced...,2.500000e+03,1. Idea: Using a straw as an electrical wire.\...,1. Idea: Using a straw as an electrical wire.,Feasibility: This is not feasible as straws ar...,1.0,1. Using a straw as a power line\nThis idea is...,1. Using a straw as a power line,This idea is quite unique as straws are not ty...,85.0
3364,13,Needle,靴下,裁縫針,1. Idea: Using a sewing needle as a sock.\nExi...,1. Idea: Using a sewing needle as a sock.,Existing products: Socks are typically priced ...,0.000000e+00,1. Idea: Using a sewing needle as a sock.\nFea...,1. Idea: Using a sewing needle as a sock.,Feasibility: This is not feasible as a sewing ...,1.0,1. Using a sewing needle as a sock\nThe idea i...,1. Using a sewing needle as a sock,The idea is to use a sewing needle as a sock. ...,90.0
238,19,Tape,文字,テープ,2. Idea: Using tape to create characters or wo...,2. Idea: Using tape to create characters or words,Existing products like stickers or decals can ...,5.000000e+02,2. Idea: Using tape to create characters or le...,2. Idea: Using tape to create characters or le...,"Feasibility: This is feasible, especially with...",75.0,2. Use tape to create characters\nUsing tape t...,2. Use tape to create characters,Using tape to create characters or words is a ...,40.0
1809,23,CD,火起こし器,ＣＤ,1. Idea: Using a CD as a fire starter\nExistin...,1. Idea: Using a CD as a fire starter,"Existing products for starting a fire, such as...",5.000000e+02,1. Idea: Using a CD as a fire starter.\nFeasib...,1. Idea: Using a CD as a fire starter.,Feasibility: This idea is not very feasible. C...,10.0,1. Idea: Using a CD as a fire starter.\nThis i...,1. Idea: Using a CD as a fire starter.,This idea is quite unique as CDs are not typic...,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,4,Newspaper,日除け,新聞,20. Idea: Using a newspaper as a sunshade.\nEx...,20. Idea: Using a newspaper as a sunshade.,Existing products: Sunshades range from 500 to...,3.000000e+02,20. Idea: Using a newspaper as a sunshade.\nCh...,20. Idea: Using a newspaper as a sunshade.,Challenges: While a newspaper can provide temp...,40.0,20. Idea: Using a newspaper as a sunshade.\nTh...,20. Idea: Using a newspaper as a sunshade.,This idea is a repetition of idea 2 and hence ...,30.0
2594,3,Rubber band,地球,輪ゴム,20. Idea: Using a rubber band as the Earth\nEx...,20. Idea: Using a rubber band as the Earth,Existing products: Globe models can range from...,0.000000e+00,20. Idea: Using a rubber band as the Earth.\nF...,20. Idea: Using a rubber band as the Earth.,Feasibility: This is not feasible as a rubber ...,1.0,20. Using a rubber band as the Earth\nThis is ...,20. Using a rubber band as the Earth,This is a highly novel idea as rubber bands ar...,100.0
2196,8,Salt,オフィスビル,塩,20. Idea: Use salt to build an office building...,20. Idea: Use salt to build an office building.,The average cost of building an office buildin...,1.000000e+09,20. Idea: Use salt to build an office building...,20. Idea: Use salt to build an office building.,Feasibility: Building an office building entir...,5.0,20. Idea: Using salt to build an office buildi...,20. Idea: Using salt to build an office building.,This is a hi

In [21]:
df_complete = pd.concat([df_eval, df_reuse])
df_complete

,Question id,Item_name,Answer,Question,value_raw,value_idea_explanation,value_explanation,value,feasibility_raw,feasibility_idea_explanation,feasibility_explanation,feasibility,novelty_raw,novelty_idea_explanation,novelty_explanation,novelty
169,19,Tape,カバー,テープ,1. Idea: Using tape as a cover\nExisting produ...,1. Idea: Using tape as a cover,Existing products like plastic covers or cloth...,1000.0,1. Idea: Using tape as a cover.\nFeasibility: ...,1. Idea: Using tape as a cover.,Feasibility: This is feasible as tape can be u...,85.0,1. Use tape as a cover\nUsing tape as a cover ...,1. Use tape as a cover,Using tape as a cover is not a very novel idea...,20.0
739,10,Drinking straw,電線,ストロー,1. Idea: Using a straw as a wire.\nExisting pr...,1. Idea: Using a straw as a wire.,Existing products: Electrical wires are priced...,2500.0,1. Idea: Using a straw as an electrical wire.\...,1. Idea: Using a straw as an electrical wire.,Feasibility: This is not feasible as straws ar...,1.0,1. Using a straw as a power line\nThis idea is...,1. Using a straw as a power line,This idea is quite unique as straws are not ty...,85.0
3364,13,Needle,靴下,裁縫針,1. Idea: Using a sewing needle as a sock.\nExi...,1. Idea: Using a sewing needle as a sock.,Existing products: Socks are typically priced ...,0.0,1. Idea: Using a sewing needle as a sock.\nFea...,1. Idea: Using a sewing needle as a sock.,Feasibility: This is not feasible as a sewing ...,1.0,1. Using a sewing needle as a sock\nThe idea i...,1. Using a sewing needle as a sock,The idea is to use a sewing needle as a sock. ...,90.0
238,19,Tape,文字,テープ,2. Idea: Using tape to create characters or wo...,2. Idea: Using tape to create characters or words,Existing products like stickers or decals can ...,500.0,2. Idea: Using tape to create characters or le...,2. Idea: Using tape to create characters or le...,"Feasibility: This is feasible, especially with...",75.0,2. Use tape to create characters\nUsing tape t...,2. Use tape to create characters,Using tape to create characters or words is a ...,40.0
1809,23,CD,火起こし器,ＣＤ,1. Idea: Using a CD as a fire starter\nExistin...,1. Idea: Using a CD as a fire starter,"Existing products for starting a fire, such as...",500.0,1. Idea: Using a CD as a fire starter.\nFeasib...,1. Idea: Using a CD as a fire starter.,Feasibility: This idea is not very feasible. C...,10.0,1. Idea: Using a CD as a fire starter.\nThis i...,1. Idea: Using a CD as a fire starter.,This idea is quite unique as CDs are not typic...,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1,Umbrella,水瓶,傘,15. Idea: Use an umbrella as a water bottle.\n...,15. Idea: Use an umbrella as a water bottle.,Existing products: Water bottles are priced ar...,1000.0,15. Idea: Using an umbrella as a water bottle....,15. Idea: Using an umbrella as a water bottle.,Feasibility: This is not feasible as an umbrel...,5.0,15. Idea: Using an umbrella as a water bottle....,15. Idea: Using an umbrella as a water bottle.,"This is a highly novel idea, as umbrellas are ...",95.0
279,1,Umbrella,パラシュート,傘,3. Idea: Using an umbrella as a parachute.\nEx...,3. Idea: Using an umbrella as a parachute.,Existing products: Parachutes can cost from 10...,50000.0,3. Using an umbrella as a parachute\nFeasibili...,3. Using an umbrella as a parachute,Feasibility: This is not feasible as an umbrel...,2.0,3. Using an umbrella as a parachute\nThis idea...,3. Using an umbrella as a parachute,"This idea is moderately novel, as umbrellas ar...",70.0
280,21,Ball pen,カッター,ボールペン,18. Idea: Using a ballpoint pen as a cutter.\n...,18. Idea: Using a ballpoint pen as a cutter.,Existing products: Cutters can range from 500 ...,1250.0,18. Idea: Using a ballpoint pen as a cutter.\n...,18. Idea: Using a ballpoint pen as a cutter.,Feasibility: This is not feasible as a ballpoi...,5.0,18. Idea: Using a ballpoint pen as a cutter.\n...,18. Idea: Using a ballpoint pen as a cutter.,"This is a somewhat novel idea, as pens are not...",60.0
281,21,Ball pen

In [22]:
df_complete.to_excel('gpt-ratings/AUT-crowdsourced-gpt4.xlsx')